In [11]:
import sys, os # for running from repo
sys.path.insert(0, os.path.abspath("../../"))

try: from brmspy import brms
except ImportError:
    %pip install -q brmspy
    from brmspy import brms

In [12]:
with brms.manage() as ctx:
    ctx.install_brms(use_prebuilt=True)

[brmspy][install_brms] Activating runtime /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0
[brmspy][install_brms] lib paths are ['/Users/sebastian/.brmspy/environment/default/Rlib', '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/library', '/Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/Rlib']
[brmspy][install_brms] Setting cmdstan path to /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/cmdstan


In [13]:
from brmspy.brms import bf, poisson, set_rescor

In [14]:
import pandas as pd
import numpy as np


epilepsy = brms.get_brms_data("epilepsy")
#epilepsy.index = [str(f"obs{i}") for i in range(len((epilepsy)))]
epilepsy.index = pd.RangeIndex(start=0, stop=len(epilepsy), step=1)
epilepsy.index = list(np.int32(v) for v in epilepsy.index)
epilepsy['Trt'] = epilepsy['Trt'].astype('category')
formula = "count ~ zAge + zBase * Trt + (1|patient)"
family = "poisson"

formula_multi = (bf(
  "count ~ zAge + zBase * Trt + visit + (1 + visit | patient)"
) + poisson()) + \
(bf(
  "Base ~ zAge + Trt + (1 | patient)"
) + poisson()) + \
set_rescor(False)

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

array (236,) int8
array [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_308b0fca",
    "size": 236,
    "content_size": 236
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


In [15]:
recall = brms.call("identity", epilepsy)


arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

In [30]:
n = 8
idx = pd.Index(range(n), name="idx")

s = pd.Series(
    pd.Categorical(
        ["a", "b", "a", None, "c", "b", "c", None][:n],
        categories=["a", "b", "c"],
        ordered=False,
    ),
    index=idx
)
t = pd.DataFrame({"cat_str": s}, index=idx)
tr = brms.call("identity", t.copy())
tr = brms.call("identity", tr)


print(t["cat_str"].dtype)
print(tr["cat_str"].dtype)
print(isinstance(t['cat_str'].dtype, pd.CategoricalDtype))
print(isinstance(tr['cat_str'].dtype, pd.CategoricalDtype))


arr int8 [ 0  1  0 -1  2  1  2 -1]
arr int8 [ 0  1  0 -1  2  1  2 -1]
category
category
True
True


arr int8 [ 0  1  0 -1  2  1  2 -1]
arr int8 [ 0  1  0 -1  2  1  2 -1]


In [17]:

print(epilepsy.dtypes)
print(recall.dtypes)

Age         float64
Base        float64
Trt        category
patient    category
visit      category
count       float64
obs        category
zAge        float64
zBase       float64
dtype: object
Age         float64
Base        float64
Trt        category
patient    category
visit      category
count       float64
obs        category
zAge        float64
zBase       float64
dtype: object


In [18]:
print(epilepsy._shm_metadata['Trt'])
epilepsy['Trt'] = epilepsy['Trt'].astype('category')
epilepsy['Trt']

{'name': 'Trt', 'np_dtype': 'int8', 'pd_dtype': 'category', 'block': {'name': 'psm_a44b8437', 'size': 236, 'content_size': 236}, 'params': {'categories': ['0', '1'], 'ordered': False, 'categories_dtype': 'object'}}
array (236,) int8
array [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_02c70bbc",
    "size": 236,
    "content_size": 236
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype":

0      0
1      0
2      0
3      0
4      0
      ..
231    1
232    1
233    1
234    1
235    1
Name: Trt, Length: 236, dtype: category
Categories (2, object): ['0', '1']

In [19]:
brms.call("identity", epilepsy)['Trt']

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

__obs_id__
0      0
1      0
2      0
3      0
4      0
      ..
231    1
232    1
233    1
234    1
235    1
Name: Trt, Length: 236, dtype: category
Categories (2, object): ['0', '1']

In [20]:
m_uni = brms.brm(
    formula=formula,
    data=epilepsy,
    family=family,
    warmup=100,
    iter=200,
    chains=4
)
id_uni = m_uni.idata

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

In [21]:
m_multi = brms.brm(
    formula=formula_multi,
    data=epilepsy,
    warmup=100,
    iter=200,
    chains=4
)
id_multi = m_multi.idata

arr int8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
arr int8 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58  0  1  2  3  4  5  6  7  8  9 10 11 12
 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58  0  1
  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 

In [22]:
import xarray as xr

m = m_uni

idata_post = brms.posterior(m).idata         # groups: posterior + constant_data
idata_obs  = brms.observed_data(m).idata     # groups: observed_data + constant_data
idata_ll   = brms.log_lik(m).idata           # groups: log_likelihood + constant_data

# optional: in-sample mean + linpred (stored in posterior)
idata_mu   = brms.posterior_epred(m).idata   # groups: posterior + constant_data   (var: count_mean)
idata_eta  = brms.posterior_linpred(m).idata # groups: posterior + constant_data   (var: count_linpred)

# 3) Compose groups that don't overlap using extend()
idata = idata_post
idata.extend(idata_obs)   # adds observed_data
idata.extend(idata_ll)    # adds log_likelihood

# 4) Compose within-group posterior variables using xarray merge
# (because extend() won't merge inside an existing group)
idata.posterior = xr.merge([idata.posterior, idata_mu.posterior, idata_eta.posterior])

print(idata.groups())
# -> ['posterior', 'observed_data', 'log_likelihood', 'constant_data']
print(idata)

['posterior', 'log_likelihood', 'observed_data', 'constant_data']
Inference data with groups:
	> posterior
	> log_likelihood
	> observed_data
	> constant_data


In [23]:
idata

Inference data with groups:
	> posterior
	> log_likelihood
	> observed_data
	> constant_data

In [24]:
sample = epilepsy.sample(322, replace=True).reset_index(drop=True).copy()

In [25]:
D = {}
for model, model_type in zip([m_uni, m_multi], ['uni', 'multi']):
    p = brms.posterior(model).idata
    ep = brms.posterior_epred(model).idata
    ep_new = brms.posterior_epred(model, sample).idata

    pp = brms.posterior_predict(model).idata
    pp_new = brms.posterior_predict(model, newdata=sample).idata
    lp = brms.posterior_linpred(model).idata
    lp_new = brms.posterior_linpred(model, sample).idata
    ll = brms.log_lik(model).idata
    ll_new = brms.log_lik(model, sample).idata

    D.update({
        f"{model_type}: posterior(m)": p,
        f"{model_type}: posterior_epred(m)": ep,
        f"{model_type}: posterior_epred(m, newdata)": ep_new,
        f"{model_type}: posterior_predict(m)": pp,
        f"{model_type}: posterior_predict(m, newdata)": pp_new,
        f"{model_type}: posterior_linpred(m)": lp,
        f"{model_type}: posterior_linpred(m, newdata)": lp_new,
        f"{model_type}: log_lik(m)": ll,
        f"{model_type}: log_lik(m, newdata)": ll_new
    })

array (322,) int8
array [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_298a2bbe",
    "size": 322,
    "content_size": 322
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


arr int8 [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
arr int8 [46 41 54  7  1 22 31 38 42 55 31 47  5  8 53 15 49 40 10  4 40 35 56 55
 41 44 36 19  6 31 50 37 24  3 26 43  6 55 49 20 32 16 57 32 58 16 26 31
  9 41 29 49 37 35 55  0 32 15 53 44 38 23 28 51 53 36 11 21 31 41 20 29
 47 51 25 25 29 51  1 56  2 33 55 16 14  7 18 39  6 36 25  0 57 53  3  5
 49 31 51 16 53 14 17 24 25 35 40 52

array (322,) int8
array [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_0fbe1fbf",
    "size": 322,
    "content_size": 322
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


arr int8 [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
arr int8 [46 41 54  7  1 22 31 38 42 55 31 47  5  8 53 15 49 40 10  4 40 35 56 55
 41 44 36 19  6 31 50 37 24  3 26 43  6 55 49 20 32 16 57 32 58 16 26 31
  9 41 29 49 37 35 55  0 32 15 53 44 38 23 28 51 53 36 11 21 31 41 20 29
 47 51 25 25 29 51  1 56  2 33 55 16 14  7 18 39  6 36 25  0 57 53  3  5
 49 31 51 16 53 14 17 24 25 35 40 52

array (322,) int8
array [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_1669a91d",
    "size": 322,
    "content_size": 322
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


...

arr int8 [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
arr int8 [46 41 54  7  1 22 31 38 42 55 31 47  5  8 53 15 49 40 10  4 40 35 56 55
 41 44 36 19  6 31 50 37 24  3 26 43  6 55 49 20 32 16 57 32 58 16 26 31
  9 41 29 49 37 35 55  0 32 15 53 44 38 23 28 51 53 36 11 21 31 41 20 29
 47 51 25 25 29 51  1 56  2 33 55 16 14  7 18 39  6 36 25  0 57 53  3  5
 49 31 51 16 53 14 17 24 25 35 40 52

array (322,) int8
array [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_cc548a97",
    "size": 322,
    "content_size": 322
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


arr int8 [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
arr int8 [46 41 54  7  1 22 31 38 42 55 31 47  5  8 53 15 49 40 10  4 40 35 56 55
 41 44 36 19  6 31 50 37 24  3 26 43  6 55 49 20 32 16 57 32 58 16 26 31
  9 41 29 49 37 35 55  0 32 15 53 44 38 23 28 51 53 36 11 21 31 41 20 29
 47 51 25 25 29 51  1 56  2 33 55 16 14  7 18 39  6 36 25  0 57 53  3  5
 49 31 51 16 53 14 17 24 25 35 40 52

array (322,) int8
array [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
create_col_meta Trt {
  "name": "Trt",
  "np_dtype": "int8",
  "pd_dtype": "category",
  "block": {
    "name": "psm_0390673b",
    "size": 322,
    "content_size": 322
  },
  "params": {
    "categories": [
      "0",
      "1"
    ],
    "ordered": false,
    "categories_dtype": "object"
  }
}


arr int8 [1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1
 0 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1
 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 1
 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0]
arr int8 [46 41 54  7  1 22 31 38 42 55 31 47  5  8 53 15 49 40 10  4 40 35 56 55
 41 44 36 19  6 31 50 37 24  3 26 43  6 55 49 20 32 16 57 32 58 16 26 31
  9 41 29 49 37 35 55  0 32 15 53 44 38 23 28 51 53 36 11 21 31 41 20 29
 47 51 25 25 29 51  1 56  2 33 55 16 14  7 18 39  6 36 25  0 57 53  3  5
 49 31 51 16 53 14 17 24 25 35 40 52

In [26]:
epilepsy.index

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       226, 227, 228, 229, 230, 231, 232, 233, 234, 235],
      dtype='int32', length=236)

In [27]:
for model, model_type in zip([m_uni, m_multi], ['uni', 'multi']):
    print("model_type:", model_type)
    print("idata:")
    for key in list(model.idata.keys()):
        print(key)
        print(model.idata[key])

        print("\n"*1)

    print("\n"*3)

model_type: uni
idata:
posterior
<xarray.Dataset> Size: 218kB
Dimensions:                  (chain: 4, draw: 100)
Coordinates:
  * chain                    (chain) int64 32B 0 1 2 3
  * draw                     (draw) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
Data variables: (12/68)
    b_Intercept              (chain, draw) float64 3kB 1.839 1.84 ... 1.73 1.62
    b_zAge                   (chain, draw) float64 3kB 0.2267 ... 0.09315
    b_zBase                  (chain, draw) float64 3kB 0.6058 0.5084 ... 0.6192
    b_Trt1                   (chain, draw) float64 3kB -0.4988 ... -0.2641
    b_zBase:Trt1             (chain, draw) float64 3kB 0.2609 0.3501 ... 0.1943
    sd_patient__Intercept    (chain, draw) float64 3kB 0.5072 0.6801 ... 0.5202
    ...                       ...
    r_patient[56,Intercept]  (chain, draw) float64 3kB 1.406 1.55 ... 1.061
    r_patient[57,Intercept]  (chain, draw) float64 3kB -0.4394 ... -0.2466
    r_patient[58,Intercept]  (chain, draw) float64 3kB -0.92

In [28]:
print("newdata shape:", sample.shape)
print("formula uni  :", formula)
print("formula multi:", formula_multi)

print("prediction outputs")
print("="*30)
for key, val in D.items():
    
    print(key)
    for k in val.groups():
        print(f"groups[{k}]")
        print(val[k])
        print("\n"*1)

newdata shape: (322, 9)
formula uni  : count ~ zAge + zBase * Trt + (1|patient)
formula multi: (bf('count ~ zAge + zBase * Trt + visit + (1 + visit | patient)') + poisson(link='log')) + (bf('Base ~ zAge + Trt + (1 | patient)') + poisson(link='log')) + set_rescor(rescor=False)
prediction outputs
uni: posterior(m)
groups[posterior]
<xarray.Dataset> Size: 218kB
Dimensions:                  (chain: 4, draw: 100)
Coordinates:
  * chain                    (chain) int64 32B 0 1 2 3
  * draw                     (draw) int64 800B 0 1 2 3 4 5 ... 94 95 96 97 98 99
Data variables: (12/68)
    b_Intercept              (chain, draw) float64 3kB 1.839 1.84 ... 1.73 1.62
    b_zAge                   (chain, draw) float64 3kB 0.2267 ... 0.09315
    b_zBase                  (chain, draw) float64 3kB 0.6058 0.5084 ... 0.6192
    b_Trt1                   (chain, draw) float64 3kB -0.4988 ... -0.2641
    b_zBase:Trt1             (chain, draw) float64 3kB 0.2609 0.3501 ... 0.1943
    sd_patient__Intercept 

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal
import datetime as dt

def make_all_pandas_dtypes_df(n: int = 8) -> pd.DataFrame:
    """
    Build a DataFrame containing a broad set of pandas column dtypes:
      - numpy scalar dtypes: bool, ints, uints, floats, complex
      - pandas extension dtypes: nullable ints, nullable bool, nullable floats, strings
      - categorical (string categories + int categories)
      - datetime64[ns], datetime64[ns, tz], timedelta64[ns]
      - period, interval
      - sparse
      - object (mixed python objects)
    """
    idx = pd.Index(range(n), name="idx")
    df = pd.DataFrame(index=idx)

    # --- numpy scalar dtypes ---
    df["np_bool"] = pd.Series(
        np.array([True, False] * (n // 2) + [True] * (n % 2), dtype=np.bool_), index=idx
    )

    df["np_int8"] = pd.Series(np.arange(n, dtype=np.int8), index=idx)
    df["np_int16"] = pd.Series(np.arange(n, dtype=np.int16) - 3, index=idx)
    df["np_int32"] = pd.Series(np.arange(n, dtype=np.int32) - 10, index=idx)
    df["np_int64"] = pd.Series(np.arange(n, dtype=np.int64) - 100, index=idx)

    df["np_uint8"] = pd.Series(np.arange(n, dtype=np.uint8), index=idx)
    df["np_uint16"] = pd.Series(np.arange(n, dtype=np.uint16) + 1, index=idx)
    df["np_uint32"] = pd.Series(np.arange(n, dtype=np.uint32) + 10, index=idx)
    df["np_uint64"] = pd.Series(np.arange(n, dtype=np.uint64) + 100, index=idx)

    df["np_float32"] = pd.Series((np.arange(n, dtype=np.float32) / 3.0), index=idx)
    df["np_float64"] = pd.Series((np.arange(n, dtype=np.float64) / 7.0), index=idx)

    df["np_complex64"] = pd.Series(
        (np.arange(n, dtype=np.float32) + 1j * np.arange(n, dtype=np.float32)).astype(
            np.complex64
        ),
        index=idx,
    )
    df["np_complex128"] = pd.Series(
        (np.arange(n, dtype=np.float64) + 1j * np.arange(n, dtype=np.float64)).astype(
            np.complex128
        ),
        index=idx,
    )

    # --- pandas nullable extension dtypes ---
    df["pd_Int64"] = pd.Series(
        [1, 2, pd.NA, 4, 5, pd.NA, 7, 8][:n], dtype="Int64", index=idx
    )
    df["pd_UInt64"] = pd.Series(
        [1, 2, pd.NA, 4, 5, pd.NA, 7, 8][:n], dtype="UInt64", index=idx
    )
    df["pd_boolean"] = pd.Series(
        [True, False, pd.NA, True, False, pd.NA, True, False][:n],
        dtype="boolean",
        index=idx,
    )
    df["pd_Float64"] = pd.Series(
        [1.5, pd.NA, 3.25, 4.0, pd.NA, 6.0, 7.0, pd.NA][:n], dtype="Float64", index=idx
    )

    # string (python-backed)
    df["pd_string"] = pd.Series(
        ["a", None, "c", "d", None, "f", "g", "h"][:n], dtype="string", index=idx
    )

    # --- datetimes / timedeltas ---
    df["np_datetime64ns"] = pd.Series(
        pd.date_range("2024-01-01", periods=n, freq="D"), index=idx
    )
    df["np_timedelta64ns"] = pd.Series(
        pd.to_timedelta(np.arange(n), unit="h"), index=idx
    )

    # tz-aware datetime (pick a non-UTC tz to catch tz dropping)
    df["pd_datetime_tz"] = pd.Series(
        pd.date_range("2024-01-01", periods=n, freq="D", tz="Europe/Tallinn"), index=idx
    )

    # --- categorical ---
    # string categories (should roundtrip without rpy2 screaming)
    df["cat_str"] = pd.Series(
        pd.Categorical(
            ["a", "b", "a", None, "c", "b", "c", None][:n],
            categories=["a", "b", "c"],
            ordered=False,
        ),
        index=idx,
    )

    # int categories (rpy2 factor conversion hates these; allowed to come back with string categories)
    df["cat_int"] = pd.Series(
        pd.Categorical(
            [1, 2, 1, None, 2, 1, None, 2][:n], categories=[1, 2], ordered=True
        ),
        index=idx,
    )

    # --- period / interval ---
    df["pd_period_M"] = pd.Series(
        pd.period_range("2024-01", periods=n, freq="M"), index=idx
    )

    intervals = pd.IntervalIndex.from_breaks(
        list(range(n + 1)), closed="left"
    )  # interval[int64, left]
    df["pd_interval"] = pd.Series(intervals[:n], index=idx)

    # --- sparse ---
    sparse_arr = pd.arrays.SparseArray(
        [0, 1, 0, 2, 0, 3, 0, 4][:n], fill_value=0, dtype=np.int64
    )
    df["pd_sparse_int64"] = pd.Series(sparse_arr, index=idx)

    # --- object (mixed python objects; should force your pickle/list-column path if you implement it) ---
    df["obj_mixed"] = pd.Series(
        [
            {"k": 1},
            (1, 2),
            Decimal("1.25"),
            dt.date(2024, 1, 1),
            None,
            b"bytes",
            {"nested": {"x": 2}},
            (3,),
        ][:n],
        dtype="object",
        index=idx,
    )

    return df


def _categories_are_strings(cat_dtype: pd.CategoricalDtype) -> bool:
    # robust check: inferred_type is "string" for string categories
    return getattr(cat_dtype.categories, "inferred_type", None) == "string"

def test_dataframe_all_dtypes(df, out):

    assert isinstance(
        out, pd.DataFrame
    ), f"Expected DataFrame back, got {type(out)!r}"
    assert list(out.columns) == list(df.columns), "Columns changed during roundtrip"
    assert len(out) == len(df), "Row count changed during roundtrip"

    failures: list[str] = []

    try:
        pd.testing.assert_index_equal(df.index, out.index)
    except Exception as e:
        failures.append(
            f"index: dtype changed: {df.index.dtype!r} -> {out.index.dtype!r}"
        )

    for col in df.columns:
        s0 = df[col]
        s1 = out[col]

        # ---- categorical special-case ----
        if isinstance(s0.dtype, pd.CategoricalDtype):
            if not isinstance(s1.dtype, pd.CategoricalDtype):
                failures.append(f"{col}: expected category, got {s1.dtype!r}")
                continue

            d0 = s0.dtype
            d1 = s1.dtype

            if bool(d0.ordered) != bool(d1.ordered):
                failures.append(
                    f"{col}: category ordered changed: {d0.ordered} -> {d1.ordered}"
                )

            codes0 = s0.cat.codes.to_numpy()
            codes1 = s1.cat.codes.to_numpy()
            if not np.array_equal(codes0, codes1):
                failures.append(
                    f"{col}: category codes changed: {codes0.tolist()} -> {codes1.tolist()}"
                )

            # If original categories are strings, require exact categories equality.
            # If original categories are non-strings (e.g., int), allow rpy2 workaround:
            # categories may come back as strings but must match str(original_categories).
            cats0 = list(d0.categories)
            cats1 = list(d1.categories)

            if _categories_are_strings(d0):
                if cats0 != cats1:
                    failures.append(
                        f"{col}: string categories changed: {cats0!r} -> {cats1!r}"
                    )
            else:
                want = [str(x) for x in cats0]
                got = [str(x) for x in cats1]
                if want != got:
                    failures.append(
                        f"{col}: non-string categories not preserved up to str(): {want!r} -> {got!r}"
                    )

            continue  # done with categorical

        # Value equality (strict)
        try:
            pd.testing.assert_series_equal(
                s0, s1, check_dtype=True, check_names=True, check_index_type=False, check_index=False
            )
        except AssertionError as e:
            failures.append(f"{col}: values changed: {e}")

    if failures:
        raise AssertionError(
            "Roundtrip identity dtype/value failures:\n- " + "\n- ".join(failures)
        )

from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

df = make_all_pandas_dtypes_df(8)

cv = make_lossy_r_converter()

with localconverter(cv) as c:
    r = c.py2rpy(df)
    out = c.rpy2py(r)

test_dataframe_all_dtypes(df, out)

/Users/sebastian/PycharmProjects/pybrms/brmspy/helpers/_rpy2/_converters/_pandas.py:126: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ns = s.view("int64")
/Users/sebastian/PycharmProjects/pybrms/brmspy/helpers/_rpy2/_converters/_pandas.py:135: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ns = s.view("int64")
/Users/sebastian/PycharmProjects/pybrms/brmspy/helpers/_rpy2/_converters/_pandas.py:124: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  ns = s.dt.tz_convert("UTC").view("int64")


TypeError: Column 'pd_period_M' dtype 'period[M]' not supported by lossy R converter. Refusing.

In [ ]:
print(df.head(3).T.to_string())

idx                                       0                          1                          2
np_bool                                True                      False                       True
np_int8                                   0                          1                          2
np_int16                                 -3                         -2                         -1
np_int32                                -10                         -9                         -8
np_int64                               -100                        -99                        -98
np_uint8                                  0                          1                          2
np_uint16                                 1                          2                          3
np_uint32                                10                         11                         12
np_uint64                               100                        101                        102
np_float32          